# Introduction to Machine Learning

# ML Megaline

This is a Machine Learning project about a Mobile carrier Megaline. They have found out that many of their subscribers use legacy plans and they want to developed a model that would analyze subscribers' behavior and recommend one of Megaline's newer plans: Smart or Ultra. With the behavior data about subscribers who have already switched to the new plans i make a ML model for this task.

imported methods and read the user behavior file as df.

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report
from sklearn.tree import DecisionTreeRegressor

df = pd.read_csv('/datasets/users_behavior.csv')

## Preprocessing

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

Got the head shape and info above to get a undrstanding of the data. Below I called descriibe to get further information.

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

I checked for missing or duplicate values above in the dataframe and it turned out to be clean.

# ML

Can we recommend a new phone plan for the subscribers on a legacy plan from their phone behavior?

## Features/Target

In [ ]:
features = df.drop('is_ultra',axis=1)
target = df['is_ultra']

## Splitting data

Below I split the dataframe into a training set, a validation set, and a test set. I split df into 80% for the test set and 20% for the validation set. I then split the train set into another 80%-20% split for the test set. used teh staratify parameter so uphold the original dataframe ratios. 

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345, stratify= target)
features_train2, features_val, target_train2, target_val = train_test_split(features_train, target_train, test_size=0.2, random_state=12345, stratify= target_train)

In [ ]:
train_shape_f = features_train.shape
test_shape_f = features_test.shape
print(f'The final model training shape is {train_shape_f} test set shape is {test_shape_f}.')

In [ ]:
train_shape_f2 = features_train2.shape
test_shape_f2 = features_val.shape
print(f'The training set shape is {train_shape_f2} for features and the Validation set shape is {test_shape_f2}.')

Found the shapes of the dataframes after the split

## Finding Best Model

The first model I tried was the RandomForestClassifier. I used a loop to check the best max depth for the range between 1-10 and best n_estimators from 100, 200, 300, 400 and 500. found the best depth to be 10 and best n_estimators to be 100.

In [ ]:
param = {'max_depth': range(1, 11), 'n_estimators': [100, 200, 300, 400, 500]}

best_params = {}
best_accuracy = 0

for depth in param['max_depth']:
    for n_estimators in param['n_estimators']:
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=n_estimators)
        model.fit(features_train2, target_train2)
        predictions_val = model.predict(features_val)
        
        accuracy = accuracy_score(target_val, predictions_val)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'max_depth': depth, 'n_estimators': n_estimators}
            
print("\nBest params:", best_params)
print("Best accuracy:", best_accuracy)


The second model I used was the DecisionTreeRegressor. I also looped through the range of 1-10 depth to find the lowest MSE. We use Mean Squared Error because accuracy_score can not be used. The best depth was 5 which gives a MSE of.15 and a .39 RMSE.

In [ ]:
best_params = {}
best_mse = 500

for depth in range(1, 11):
    model = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model.fit(features_train2, target_train2)
    predictions_val = model.predict(features_val)
        
    mse = mean_squared_error(target_val, predictions_val)
        
    if mse < best_mse:
        best_mse = mse
        best_params = {'max_depth': depth}
            
print('\nBest params:', best_params)
print('Best MSE:', best_mse)
print('RMSE', best_mse**.5)

### Best model

In [ ]:
best_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=100)
best_model.fit(features_train, target_train)
best_predictions = best_model.predict(features_test)

## Sanity check

### RMSE

To sanity check the model I found the RMSE.

In [ ]:
mean_squared_error(target_val, predictions_val)**.5

### Accuracy

Checked for the accuracy of the model to verify its validity. With the test set it came out to be 80%.

In [ ]:
print("Accuracy:", accuracy_score(target_test, best_predictions))
print()
print("Classification Report:")
print(classification_report(target_test, best_predictions))

For this project I am looking to recommend a new phone plan for users that are on a legacy plan.Before working with the data I did preprocessing to check for missing or duplicate values as well as the overall structure of the dataframe to make sure it was ready for further processing in teh ML model. In order to make this I did a ML model with a training set of 80% and 20% for the validation set to build up my model. I then split the 20% into another 80%-20% split for the test set. 

I tested a range of hyperprameters to find the best model for the test set, which ended up boing a max_depth of 8 and n_estimators of 100. I then did a sanity check by finding the RMSE. The RMSE measures the average difference between the model’s predicted values and the actual values which ended up being .46 for its value. To ensure the accuracy of the model I found the accuracy score of the model which ended up at 81%.